In [4]:
from dotenv import load_dotenv
import os
import base64
from requests import post,get
import json


load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic "+ auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query
    result= get(query_url, headers = headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("No artist with this name exists")
        return None
    
    return json_result[0]

def get_songs_recommendations(token, artist_id, min_popularity, max_popularity):
    url = f"https://api.spotify.com/v1/recommendations?limit=100&market=US&seed_artists={artist_id}&min_popularity={min_popularity}&max_popularity={max_popularity}"
    headers = get_auth_header(token)
    result = get(url, headers= headers)
    if result.status_code ==200:
        json_result=json.loads(result.content)
        return json_result['tracks']
    else:
        print(f"Failed to fetch recommendations. Status code: {result.status_code}")
        return None


In [3]:
def search_for_song(token, song_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={song_name}&type=track&limit=1"
    query_url = url + query
    result= get(query_url, headers = headers)
    if result.status_code == 200:
        json_result = json.loads(result.content)
        return json_result
    else:   
        print(f"Failed to find song. Status code: {result.status_code}")
        return None
    
    
    

In [4]:
token = get_token()


In [5]:
song_name = input("What song do you want?")

In [6]:
answer = search_for_song(token, song_name)

In [9]:

answer['tracks']['items'][0]['id']

'5TRPicyLGbAF2LGBFbHGvO'

In [12]:

answer['tracks']['items'][0]['album']['images'][1]['url']

'https://i.scdn.co/image/ab67616d00001e0226f7f19c7f0381e56156c94a'

In [10]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
def get_token():
    cid = os.getenv("CLIENT_ID")
    secret =  os.getenv("CLIENT_SECRET")
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    return sp


In [2]:
import pandas as pd


def song_id(song):
    sp = get_token()
    song_id = sp.search(q=song, type = 'track', limit = 1)
    song_id = song_id['tracks']['items'][0]['id']

    track = sp.track(song_id)
    track_features = sp.audio_analysis(song_id)

    track_df = pd.DataFrame(track)
    track_features_df = pd.DataFrame(track_features)

    track_final_df = pd.merge(track_df, track_features_df, on='id')

    track_final= track_final_df.tolist()

    data = track_final

    return data


In [14]:
sp = get_token()
song_id = sp.search(q='Flashing Lights', type = 'track', limit = 1)
song_id = song_id['tracks']['items'][0]['id']

song_id


'5TRPicyLGbAF2LGBFbHGvO'

In [45]:

track = sp.track(song_id)
track_dict = {
    'song': track['name'],
    'artist': track['album']['artists'][0]['name'],
    'popularity': track['popularity'],
    'uri': track['uri']
}
track_features = sp.audio_features([song_id])

track_df = pd.DataFrame([track_dict])
track_df


,song,artist,popularity,uri
0,Flashing Lights,Kanye West,88,spotify:track:5TRPicyLGbAF2LGBFbHGvO


In [46]:
track_features_df = pd.DataFrame(track_features)
track_features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.639,0.628,6,-7.578,0,0.0399,0.0381,0,0.386,0.43,90.482,audio_features,5TRPicyLGbAF2LGBFbHGvO,spotify:track:5TRPicyLGbAF2LGBFbHGvO,https://api.spotify.com/v1/tracks/5TRPicyLGbAF...,https://api.spotify.com/v1/audio-analysis/5TRP...,237507,4


In [48]:
track_final_df = pd.merge(track_df, track_features_df, on = 'uri')
track_final_df['duration'] = track_final_df['duration_ms']/1000
track_final_df

,song,artist,popularity,uri,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,duration
0,Flashing Lights,Kanye West,88,spotify:track:5TRPicyLGbAF2LGBFbHGvO,0.639,0.628,6,-7.578,0,0.0399,...,0.386,0.43,90.482,audio_features,5TRPicyLGbAF2LGBFbHGvO,https://api.spotify.com/v1/tracks/5TRPicyLGbAF...,https://api.spotify.com/v1/audio-analysis/5TRP...,237507,4,237.507


In [49]:
track_final_df = track_final_df[['acousticness','artist', 'danceability','duration', 'energy', 'instrumentalness', 'liveness', 
                                 'loudness', 'popularity', 'song', 'speechiness', 'tempo', 'valence']]
track_final_df

,acousticness,artist,danceability,duration,energy,instrumentalness,liveness,loudness,popularity,song,speechiness,tempo,valence
0,0.0381,Kanye West,0.639,237.507,0.628,0,0.386,-7.578,88,Flashing Lights,0.0399,90.482,0.43
